# Purpose: input raw data files from Kaggle and output cleaned data file as csv format for analysis in later parts. 

This is to ensure that we do not have to redo the data cleaning steps everytime we perform data analysis

## Import packages

In [1]:
import pandas as pd
import numpy as np
import glob
import re
import os
import fitz
import math
from textblob import TextBlob
from langdetect import detect

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Lu
[nltk_data]     Xinyi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#read data
trump = pd.read_csv('data/hashtag_donaldtrump.csv', lineterminator='\n')
biden = pd.read_csv('data/hashtag_joebiden.csv', lineterminator='\n')

In [3]:
trump.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:01,1.316529e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,3.316176e+08,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,1067661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530
2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
3,2020-10-15 00:00:02,1.316529e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,8.283556e+17,Trumpytweeter,trumpytweeter,"If he doesn't tweet for some time, should we b...",...,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591
4,2020-10-15 00:00:08,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,4.741380e+07,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121


In [4]:
biden.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:18,1.316529e+18,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,Twitter for iPad,8.099044e+08,Cheri A. 🇺🇸,Biloximeemaw,"Locked and loaded Meemaw. Love God, my family ...",...,6628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.517827283
2,2020-10-15 00:00:20,1.316529e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3.494182e+09,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
3,2020-10-15 00:00:21,1.316529e+18,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,Twitter for iPhone,8.242596e+17,Michelle Ferg,MichelleFerg4,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.553481849
4,2020-10-15 00:00:22,1.316529e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032807e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132


In [5]:
trump.shape

(970919, 21)

In [6]:
biden.shape

(776886, 21)

In [7]:
trump.dtypes

created_at               object
tweet_id                float64
tweet                    object
likes                   float64
retweet_count           float64
source                   object
user_id                 float64
user_name                object
user_screen_name         object
user_description         object
user_join_date           object
user_followers_count    float64
user_location            object
lat                     float64
long                    float64
city                     object
country                  object
continent                object
state                    object
state_code               object
collected_at             object
dtype: object

In [8]:
trump.country.unique()

array(['United States of America', nan, 'United Kingdom', 'Germany',
       'United States', 'Australia', 'Costa Rica', 'Barbados', 'Cameroon',
       'The Netherlands', 'Mexico', 'Canada', 'France', 'Peru',
       'Argentina', 'South Korea', 'Finland', 'Mongolia', 'India',
       'Japan', 'Turkey', 'Switzerland', 'Iran', 'Portugal', 'Ireland',
       'Russia', 'Greece', 'Egypt', 'Ecuador', 'Colombia', 'Sri Lanka',
       'Italy', 'China', 'Malaysia', 'Indonesia', 'Venezuela', 'Thailand',
       'Sweden', 'Nepal', 'Bangladesh', 'Philippines',
       'Dominican Republic', 'New Zealand', 'Chile', 'Honduras',
       'Azerbaijan', 'Georgia', 'Spain', 'Cyprus', 'Morocco',
       'Netherlands', 'Saudi Arabia', 'Norway', 'Singapore', 'Cuba',
       'Niger', 'Hungary', 'Greenland', 'United Arab Emirates', 'Brazil',
       'Uganda', 'Guatemala', 'Pakistan', 'Taiwan', 'Qatar',
       'Afghanistan', 'South Africa', 'Nigeria', 'Slovakia', 'Guyana',
       'Kenya', 'Iraq', 'Denmark', 'Uruguay', 'Be

In [9]:
trump.continent.unique()

array(['North America', nan, 'Europe', 'Oceania', 'Africa',
       'South America', 'Asia', 'Antarctica'], dtype=object)

In [10]:
biden.continent.unique()

array(['North America', nan, 'Europe', 'Asia', 'Africa', 'Oceania',
       'South America', 'Antarctica'], dtype=object)

In [25]:
def CleanData(data):
    df = data.copy()
    
    #only filter for english tweets
    for index, row in df.iterrows():
        try:
            language = detect(row['tweet'])
        except:
            language = 'error'
        df.loc[index, 'tweet_lang'] = language
        
    df = df[df['tweet_lang'] == 'en']
    
    #drop rows with NA country as we want to analyse by country
    df = df.dropna(subset=['country'])

    #convert column to datetime types
    date_cols = ['created_at', 'user_join_date', 'collected_at']
    numeric_cols = ['tweet_id', 'likes', 'retweet_count', 'user_id', 'user_followers_count']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    for col in numeric_cols:
        df[col] = df[col].astype(int)
    
    #clean up country names
    df['country'] = df['country'].replace('United States of America', 'United States')
    df = df.reset_index(drop = True)
    
    #drop irrelevant columns
    df = df.drop(columns = ['tweet_id', 'source', 'user_id', 'user_name', 'user_screen_name', 'user_description', 'user_join_date', 'collected_at'])
    #clean up tweets
    #remove punctuation and numbers
    punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for index, row in df.iterrows():
        tweet = row['tweet']
        tweet = ''.join([char for char in tweet if char not in punctuation])
        tweet = re.sub('[0-9]+', '', tweet)
        df.loc[index, 'tweet_new'] = tweet
    
    #tokenization
    df['tweet_new'] = df['tweet_new'].apply(lambda x: re.split('\W+', x.lower()))
    
    #remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    df['tweet_new'] = df['tweet_new'].apply(lambda x: [word for word in x if word not in stopwords])
    
    #stemming and lemmatization
    ps = nltk.PorterStemmer()
    wn = nltk.WordNetLemmatizer()
    df['tweet_new'] = df['tweet_new'].apply(lambda x: [ps.stem(word) for word in x])
    df['tweet_new'] = df['tweet_new'].apply(lambda x: [wn.lemmatize(word) for word in x])
    return df

### Sampling: 10000 rows from each trump/biden dataset 3 days before the election & 3 days after the election

In [20]:
# get data 3 days before election & 3 days after election; election day was nov 3
trump_pre = trump.loc[(trump['created_at'] < '2020-11-03') & (trump['created_at'] >= '2020-10-27')]
trump_post = trump.loc[(trump['created_at'] > '2020-11-04') & (trump['created_at'] <= '2020-11-10')]

biden_pre = biden.loc[(biden['created_at'] < '2020-11-03') & (biden['created_at'] >= '2020-10-27')]
biden_post = biden.loc[(biden['created_at'] > '2020-11-04') & (biden['created_at'] <= '2020-11-10')]

# sample 10000 rows from each dataset
trump_pre = trump_pre.sample(n = 10000)
trump_post = trump_post.sample(n = 10000)
biden_pre = biden_pre.sample(n = 10000)
biden_post = biden_post.sample(n = 10000)

print(trump_pre.shape, trump_post.shape)
print(biden_pre.shape, biden_post.shape)

df_pre = pd.concat([trump_pre, biden_pre])
df_post = pd.concat([trump_post, biden_post])

print(df_pre.shape)
print(df_post.shape)

(10000, 21) (10000, 21)
(10000, 21) (10000, 21)
(20000, 21)
(20000, 21)


In [21]:
df_pre_cleaned = CleanData(df_pre)
df_pre_cleaned.shape

(7348, 23)

In [22]:
df_post_cleaned = CleanData(df_post)
df_post_cleaned.shape

(5433, 23)

In [23]:
df_pre_cleaned.to_excel('sample_pre.xlsx')
df_post_cleaned.to_excel('sample_post.xlsx')

### Without sampling

In [29]:
full_trump_pre = trump.loc[trump['created_at'] < '2020-11-03']
full_biden_pre = biden.loc[biden['created_at'] < '2020-11-03']

full_trump_pre.tail()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
458180,2020-11-02 23:59:53,1.323415e+18,@BuzzFeedNews What's the difference between #t...,0.0,0.0,Twitter for Android,1.201329e+18,Bertille,ublezmvrarlzcv1,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-04 10:20:22.897914
458181,2020-11-02 23:59:55,1.323415e+18,#Trump #AmericaFirst #4MoreYears https://t.co/...,0.0,0.0,Twitter for iPhone,5.395178e+07,BendTheBar,FortNeyland95,Lead Engineer/#BES admin /#Father/#powerliftin...,...,1056.0,U.S.A.,31.193277,30.054019,NaN,Egypt,Africa,Alexandria Governorate,NaN,2020-11-03 14:23:58.069086
458182,2020-11-02 23:59:56,1.323415e+18,#BidenHarris2020 is a corrupt ticket it’s goin...,0.0,0.0,Twitter for iPhone,2.970398e+09,Stephen,SCEaker,business owner,...,51.0,"Palm Beach, FL",26.627980,-80.449417,NaN,United States of America,North America,Florida,FL,2020-11-04 10:56:44.117655
458183,2020-11-02 23:59:57,1.323415e+18,#Bolton: #Trump trying to set up con on his ow...,0.0,0.0,PulpNews,1.009870e+08,PulpNews Crime,PulpNews,The Fastest Crime News Updates on the Planet!,...,4978.0,USA,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,2020-11-03 14:23:57.995672
458184,2020-11-02 23:59:58,1.323415e+18,@sarahfrmm @NellyUziVert #cancelnel #kpop #tru...,1.0,0.0,Twitter for iPhone,1.225617e+18,fetty wap enthusiast,pplcallmebubba,17 | adryan | mexican and shit | i'm not reall...,...,245.0,limbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-04 10:56:43.805852


In [30]:
full_biden_pre.tail()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
311170,2020-11-02 23:59:43,1.323415e+18,#Election2020 #LadyGaga #JoeBiden https://t.c...,1.0,1.0,Twitter for iPhone,1.217672e+18,Jesus Lopez,xxdannyj92,NaN,...,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-03 14:30:19.041515
311171,2020-11-02 23:59:46,1.323415e+18,Every vote matters. Don't let yours go wasted....,0.0,0.0,Twitter for Android,9.209848e+17,voidheads.com,voidheads,• Entertainment Channel\n- DM for collaboratio...,...,10.0,"Bengaluru, India",12.979120,77.591300,Bengaluru,India,Asia,Karnataka,KA,2020-11-03 14:32:32.011901
311172,2020-11-02 23:59:50,1.323415e+18,"#US #PresidentialElection Roadmap, #Prediction...",0.0,0.0,Twitter Web App,2.838224e+08,Keystone Speculator,KeySpeculator,Simple and easy to understand technical analys...,...,541.0,"Laurel Highlands, Pennsylvania",40.360755,-78.975559,Lower Yoder Township,United States of America,North America,Pennsylvania,PA,2020-11-03 14:32:31.604507
311173,2020-11-02 23:59:53,1.323415e+18,@BuzzFeedNews What's the difference between #t...,0.0,0.0,Twitter for Android,1.201329e+18,Bertille,ublezmvrarlzcv1,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-03 14:32:31.647706
311174,2020-11-02 23:59:58,1.323415e+18,@sarahfrmm @NellyUziVert #cancelnel #kpop #tru...,1.0,0.0,Twitter for iPhone,1.225617e+18,fetty wap enthusiast,pplcallmebubba,17 | adryan | mexican and shit | i'm not reall...,...,243.0,limbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-03 14:32:31.570968


In [31]:
print(full_trump_pre.shape)
print(full_biden_pre.shape)

(458185, 21)
(311175, 21)


In [32]:
#10000 rows take 2 mins to clean
#100000 rows take 48 mins to clean
#sample 50000 rows 
sample_trump_pre = full_trump_pre.sample(n = 50000)
sample_biden_pre = full_biden_pre.sample(n = 50000)
print(sample_trump_pre.shape)
print(sample_biden_pre.shape)

(50000, 21)
(50000, 21)


In [34]:
sample_trump_pre.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
454155,2020-11-02 22:23:12,1.323390e+18,"#Trump o #Biden, chi vincerà?\n\nhttps://t.co/...",4.0,0.0,Twitter for Android,9.923355e+17,The Cryptonomist Italia,cryptonomist_,Raccontiamo il futuro.\nLe notizie più esclusi...,...,2780.0,Svizzera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-04 10:21:04.085146
126175,2020-10-21 13:00:00,1.318900e+18,@JohnBarlowe @Steeldigital @durante66 @CaesarS...,0.0,0.0,Twitter for iPhone,3.947188e+07,KathyB,kittylolas,"Believing in equality, fairness, disinterested...",...,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27 18:00:23.815929
224401,2020-10-24 20:22:00,1.320098e+18,@nytimes The fascist @SenateGOP talk a good ga...,2.0,0.0,Twitter for Android,8.094018e+17,Mike VOTE Magrath🌊🌊🌊,realMikeMagrath,"#VoteBlue #Resist #FBR #ANTIFA Businessman, Pa...",...,2919.0,United States,39.78373,-100.445882,NaN,United States,North America,NaN,NaN,2020-10-27 17:42:26.691611
156448,2020-10-22 21:57:07,1.319397e+18,@Fermitthakrog8 @KingBelloIII1 @CNN I was bein...,0.0,0.0,Twitter for iPhone,1.234975e+18,David F. Dodge,DavidFDodge1,"Emergency Services Program Assistant, Benefits...",...,5079.0,"Le Roy, NY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27 17:54:11.036826
292430,2020-10-28 02:08:09,1.321273e+18,@RudyGiuliani is done touching himself 2 the k...,1.0,0.0,Twitter for iPhone,2.975658e+09,Len,plzhelpkids,NaN,...,144.0,"Anywhere, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-29 17:21:07.505558


In [35]:
sample_biden_pre.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
306645,2020-11-02 20:50:15,1.323367e+18,@TotallyNot_Aj #BidenHarris2020 #AmericasGreat...,1.0,0.0,Twitter for iPhone,1.319333e+18,SLIME,LuverBo00993632,im not sad anymore 🤝,...,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-03 14:33:10.215091
112944,2020-10-23 01:30:18,1.319451e+18,watch #JoeBiden eyes you can tell he's reading...,0.0,0.0,Twitter for Android,5.845191e+08,doogiƎ,doogi3ville,we1come 2 doogi3ville delayed by corona. i am ...,...,1578.0,urf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27 18:10:30.699015
296827,2020-11-02 14:27:22,1.323270e+18,@EricTrump @realDonaldTrump Trump is willing t...,0.0,0.0,Twitter Web App,7.557980e+08,oxforduk,Oxforduk23,"Mask-wearing, Biden-supporting suburbanite see...",...,402.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-03 14:34:39.334432
68026,2020-10-20 04:55:47,1.318416e+18,#Trump has repeatedly made baseless claims tha...,0.0,0.0,Twitter for iPhone,1.026216e+18,Webb Art,Art1Molly,"Artist.Graduated from UC Santa Cruz-Jewelry, A...",...,124.0,United States,39.78373,-100.445882,NaN,United States,North America,NaN,NaN,2020-10-21 10:01:17.026808349
46254,2020-10-18 04:34:11,1.317685e+18,I’m not sure #JoeBiden knows what the truth is...,0.0,0.0,Twitter for iPhone,7.138470e+07,sorrowen💀💀,sorrowen,"Aspiring comic artist,Christian,conservative,s...",...,898.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 06:48:05.463077835


In [36]:
%%time
cleaned_trump_pre = CleanData(sample_trump_pre)
cleaned_trump_pre.head()

Wall time: 20min 29s


,created_at,tweet,likes,retweet_count,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,tweet_lang,tweet_new
0,2020-10-24 20:22:00,@nytimes The fascist @SenateGOP talk a good ga...,2,0,2919,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,en,"[nytim, fascist, senategop, talk, good, game, ..."
1,2020-11-01 15:46:25,Arab America Poll: Joe Biden Leads Donald Trum...,1,1,5498,"Washington, DC",38.894992,-77.036558,Washington,United States,North America,District of Columbia,DC,en,"[arab, america, poll, joe, biden, lead, donald..."
2,2020-10-27 15:29:30,@SteveHofstetter #Trump Don't forget your T-sh...,1,0,4521,"Los Angeles, CA",34.053691,-118.242766,Los Angeles,United States,North America,California,CA,en,"[stevehofstett, trump, dont, forget, tshirt, t..."
3,2020-10-30 16:25:47,🚨 MAIL IN BALLOT ALERT🚨\n\n#Minnesota\nMOVED T...,0,0,2910,"Las Vegas, NV BWO Detroit, MI",42.331430,-83.045750,NaN,United States,North America,Michigan,MI,en,"[, mail, ballot, alert, minnesota, move, goalp..."
4,2020-10-17 04:51:16,So #Trump “sorta kinda” admitted that he wasn’...,2,0,2451,New York,40.712728,-74.006015,New York,United States,North America,New York,NY,en,"[trump, sorta, kinda, admit, test, day, st, de..."


In [37]:
%%time
cleaned_biden_pre = CleanData(sample_biden_pre)
cleaned_biden_pre.head()

Wall time: 24min 42s


,created_at,tweet,likes,retweet_count,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,tweet_lang,tweet_new
0,2020-10-20 04:55:47,#Trump has repeatedly made baseless claims tha...,0,0,124,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,en,"[trump, repeatedli, made, baseless, claim, ele..."
1,2020-11-01 01:24:39,I remade #billyjoel’s #WeDidntStartTheFire w/l...,0,0,872,NYC,40.712728,-74.006015,New York,United States,North America,New York,NY,en,"[remad, billyjoel, wedidntstartthefir, wlyric,..."
2,2020-10-21 08:03:40,#NSTworld Trump has spent recent days promotin...,2,2,726128,Malaysia,4.569375,102.265682,NaN,Malaysia,Asia,NaN,NaN,en,"[nstworld, trump, spent, recent, day, promot, ..."
3,2020-11-02 23:09:35,On today's 2020Talks podcast - As the calendar...,2,0,87031,San Francisco,37.779026,-122.419906,San Francisco,United States,North America,California,CA,en,"[today, talk, podcast, calendar, turn, novemb,..."
4,2020-11-02 19:38:23,Black Lives Matter To Biden!!!\n\nBlack Lives ...,5,1,7362,"Kentucky, USA",37.572603,-85.155141,NaN,United States,North America,Kentucky,KY,en,"[black, live, matter, biden, black, live, matt..."


In [38]:
cleaned_trump_pre.to_excel('sample_trump_pre.xlsx')

In [39]:
cleaned_biden_pre.to_excel('sample_biden_pre.xlsx')